<a href="https://colab.research.google.com/github/StevenVuong/MSc_Project/blob/master/misc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal: Build a joint deeplearning model which trains on ages and genders, to classify that of the fifth batch.** 
**Then ensemble this with deep learning prediction outputs of our best model to get the final classification**

In [0]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# other imports to handle files
import os
import pickle
import csv

# deep learning imports
from keras.models import Sequential, load_model, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, Convolution1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, generic_utils, to_categorical
from keras.layers import LeakyReLU, Input, ReLU, concatenate
from keras import regularizers

# to split our dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# to mount our drive
from google.colab import drive

Using TensorFlow backend.


In [0]:
# mount google drive into google colab
drive.mount('/content/gdrive')

os.chdir('/content')

# go to where we will be working
print (os.listdir())
os.chdir('gdrive/My Drive/msc_project/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
['.config', 'gdrive', 'sample_data']


**Run through our batches and train our model after building it**

In [0]:
## Loop through all our total slices and accumulate all the total slices info
training_batch_f = os.listdir('stored_batches_trim')[:6]
print (training_batch_f)

# build ultima total slices info
tsi_ultima = []

# load the pickle (train with 0 to 6)
for tbf in training_batch_f:
  with open('stored_batches_trim/'+tbf, 'rb') as f:
    total_slices, total_slices_info = pickle.load(f) # stored_batches/total_slices_batch5
    
    tsi_ultima.extend(total_slices_info)
    
print (np.shape(tsi_ultima))

['dtotal_slices_batch0.pkl', 'dtotal_slices_batch1.pkl', 'dtotal_slices_batch2.pkl', 'dtotal_slices_batch3.pkl', 'dtotal_slices_batch4.pkl', 'dtotal_slices_batch6.pkl']
(620, 4)


In [0]:
np.shape(tsi_ultima) # use this to train

(620, 4)

In [0]:
def get_classification(total_slices_info):
    '''get information of patient from total slices info'''
    classif = [s[2] for s in total_slices_info]
    classif = np.array(to_categorical(classif, 2))
    
    # now do the same for the sex, 'F' is 0, 'M' is 1
    sex = [s[1] for s in total_slices_info]
    for i in range(len(sex)):
        if sex[i] == 'F':
            sex[i] = 0
        if sex[i] == 'M':
            sex[i] = 1
    sex = np.array(to_categorical(sex, 2))
    
    # finally for age, one hot encode ages 0 to 100 (101 classes then)
    ages = [s[3] for s in total_slices_info]
    ages = np.array(to_categorical(ages, 101))
    
    return classif, sex, ages

y_values_train, sex_train, ages_train = get_classification(tsi_ultima)

In [0]:
def create_mlp(dim, regress=False):
    '''Create our MLP network'''
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(2, activation="relu"))
 
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
 
    # return our model
    return model

In [0]:
# creds: https://www.puzzlr.org/the-keras-functional-api-five-simple-examples/
# Create MLP models
mlp_sex = create_mlp(sex_train.shape[1], regress=False)
mlp_age = create_mlp(ages_train.shape[1], regress=False)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combined_input = concatenate([mlp_sex.output, mlp_age.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(8, activation="relu")(combined_input)
x = Dense(2, activation="sigmoid")(x)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
as_model = Model(inputs=[mlp_sex.input, mlp_age.input], outputs=x)

In [0]:
# compile our model
as_model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-3, decay=1e-3 / 200), 
              metrics = ['categorical_accuracy']) # decay in Adam..

# train the model
print("training model...")
as_model.fit([sex_train, ages_train], y_values_train, validation_split=0.1, epochs=200, batch_size=205)

In [0]:
# load our pickle model (ALTERNATIVE)
with open('all_mprage_grappa/z_tests/as_model.pkl', 'rb') as f: # also 'total_slices_all.pkl' ## RENAMED 5 TO 7, TESTING IT
  as_model = pickle.load(f) # stored_batches/total_slices_batch5

In [0]:
# load our pickle batch of data
with open('stored_batches_trim/dtotal_slices_batch5.pkl', 'rb') as f: # also 'total_slices_all.pkl' ## RENAMED 5 TO 7, TESTING IT
  total_slices_test, total_slices_info_test = pickle.load(f) # stored_batches/total_slices_batch5
  
# get the test factors
y_values_test, sex_test, ages_test = get_classification(total_slices_info_test)

# expand the set
total_slices_test = np.array(total_slices_test)
total_slices_test = np.expand_dims(total_slices_test, axis=4)

In [0]:
# test our model against the hidden one
score, acc = as_model.evaluate([sex_test, ages_test], y_values_test)

print ("Score: %.2f" % score) # just by running a deep learning model with the goddamn ages and gender, wtf

102/102 [==============================] - 0s 2ms/step
Score: 0.98


In [0]:
## save 98% model 
# pickle.dump( as_model, open( "all_mprage_grappa/z_tests/as_model.pkl", "wb" ) )

In [0]:
as_preds_train = as_model.predict([sex_train, ages_train]) # this is what we will feed into next model
as_preds_test = as_model.predict([sex_test, ages_test]) # this is what we will use when testing

**Next part, get predictions from model on ages and sex and predictions from images, feed into another deep NN to try to predict the output, then test accuracy**

In [0]:
# ALTERNATIVELY: load our cnn_models
cnn_model = load_model('all_mprage_grappa/z_tests/models/_all_batches_040.h5') # take 040, best performing one so far

W0620 06:49:10.729431 140591951792000 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#load batch slices, predict and aggregate scores
cnn_train_preds = []

# load the pickle (train with 0 to 6)
print (training_batch_f)
for tbf in training_batch_f:
  with open('stored_batches_trim/'+tbf, 'rb') as f:
    total_slices, total_slices_info = pickle.load(f) 
    # predict and aggregate
    total_slices = np.array(total_slices)
    total_slices = np.expand_dims(total_slices, axis=4)
    
    # split and predict, too many to handle otherwise and will crash
    sub_arrays = np.array_split(total_slices, 50)
    
    for i in range(len(sub_arrays)):
      
      sub_array = sub_arrays[i]
      cnn_prediction = cnn_model.predict(sub_array)
      cnn_train_preds.extend(cnn_prediction)
    
print (np.shape(cnn_train_preds))

W0619 14:07:23.063912 140617590118272 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


['dtotal_slices_batch0.pkl', 'dtotal_slices_batch1.pkl', 'dtotal_slices_batch2.pkl', 'dtotal_slices_batch3.pkl', 'dtotal_slices_batch4.pkl', 'dtotal_slices_batch6.pkl']
(620, 2)


**Now make deep learning framework where we load our cnn train preds, our as train preds and aggregate**

In [0]:
cnn_train_preds = pickle.load( open( "all_mprage_grappa/z_tests/cnn_training_preds.pkl", "rb" ) )
cnn_train_preds = np.array(cnn_train_preds)

as_preds_train = np.array(as_preds_train)

y_train = y_values_train

In [0]:
def to_binary(cat_array):
  '''Function to convert categorical back to binary values'''
  binary_output_array = []
  for i in range(len(cat_array)):
    binary_output_array.append(np.argmax(cat_array[i]))
    
  binary_output_array = np.array(binary_output_array)
  return binary_output_array

In [0]:
# Create MLP models
mlp_cnn = create_mlp(2, regress=False) # value of 1 for binary, 2 for categorical
mlp_as = create_mlp(2, regress=False)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combined_input_two = concatenate([mlp_cnn.output, mlp_as.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
z = Dense(4, activation="relu")(combined_input_two)
z = Dropout(0.2)(z)
z = Dense(2, activation="sigmoid")(z)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
combined_model = Model(inputs=[mlp_cnn.input, mlp_as.input], outputs=z)

In [0]:
# compile our model
combined_model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-3, decay=1e-3 / 200), 
              metrics = ['categorical_accuracy']) # decay in Adam..

# train the model
print("training model...")
combined_model.fit([cnn_train_preds, as_train_preds], y_values_train, validation_split=0.1, epochs=200)

In [0]:
# pickle.dump( combined_model, open( "all_mprage_grappa/z_tests/combined_model_binary.pkl", "wb" ) )

**Load last one and test it out**

In [0]:
# get predictions of cnn_model on test data
# get predictions of as_model on test data
# input these predictions into our combined_model
# evaluate these against the actual y-value, pray it is not overfitted...

(102, 2)

In [0]:
## Getting predictions of cnn model on test data

# Split into smaller chunks so our GPU can handle it
sub_arrays = np.array_split(total_slices_test, 51)

cnn_preds_test = []
# run through model and test
for i in range(len(sub_arrays)):
  sub_array = sub_arrays[i]

  # got our scoring metric off the back of this too, all in one notebook
  cnn_pred = cnn_model.predict(sub_array)
  cnn_preds_test.extend(cnn_pred)

In [0]:
cm_score, cm_acc = combined_model.evaluate([cnn_preds_test, as_preds_test], y_values_test)

print ("Combined model score: %.2f, and combined model accuracy: %.2f" % (cm_score, cm_acc))

102/102 [==============================] - 0s 129us/step
Combined model score: 1.08, and combined model accuracy: 0.59


**Try: XGboosting the training predicted outputs**

In [0]:
from xgboost import XGBClassifier
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score

In [0]:
# convert everything to binary
cnn_train_preds_binary = to_binary(cnn_train_preds)
as_train_preds_binary = to_binary(as_preds_train)
y_train_binary = to_binary(y_train)

cnn_test_preds_binary = to_binary(cnn_preds_test)
as_test_preds_binary = to_binary(as_preds_test)
y_test_binary = to_binary(y_values_test)

In [0]:
np.shape(y_train_binary) # no worky

(620,)

In [0]:
# make into a matrix and input this into our space

combined_zipped_train = list(zip(cnn_train_preds_binary, as_train_preds_binary))
combined_zipped_train = np.array(combined_zipped_train)

# fit model with training data
xgb_model = XGBClassifier()
xgb_model.fit(combined_zipped_train, y_train)

In [0]:
combined_zipped_test = list(zip(cnn_preds_test, as_preds_test))
combined_zipped_test = np.array(combined_zipped_test)

y_values_predictions = []
for i in range(len(combined_zipped_test)):
  cz_test_csr = csr_matrix(combined_zipped_test[i])
  xgb_preds = xgb_model.predict(cz_test_csr)
  y_values_predictions.append(xgb_preds)
  
y_values_predictions = np.array(y_values_predictions)

In [0]:
accuracy_score(y_values_test, y_values_predictions, normalize=True)

0.0

In [0]:
cnn_train_preds # train with this as an input
as_preds_train # another input
y_train # ideal output

#test
cnn_preds_test
as_preds_test
y_values_test


To try: reverse classification on each model and feed these in, see how our model holds up. Then do the same for our test and compare